In [30]:
from github import Github, Auth
from dotenv import dotenv_values
from tqdm import tqdm
import pandas as pd
from itertools import chain, map, filterfalse

config = dotenv_values(".env")

g = Github(auth=Auth.Token(config['GITHUB_ACCESS_TOKEN']))


In [31]:
label_map = {
	"facebook/react": {
		"Type: Bug": "bug",
		"Type: Feature Request": "feature",
		"Type: Question": "question",
	},
}


In [34]:

def issues_by_repo(repo_full_name: str):
	repo_labels = set(label_map[repo_full_name].keys())
	repo = g.get_repo(repo_full_name)
	for label in repo_labels:
		for issue in repo.get_issues(state="closed", labels=[label]):
			issue_labels = set([label.name for label in issue.labels])
			matching_labels = repo_labels.intersection(issue_labels)
			if len(matching_labels) != 1:
				continue
			standard_label = label_map[repo_full_name][matching_labels.pop()]
			yield [repo_full_name, standard_label, issue.title, issue.body]

pd.DataFrame(data=tqdm(issues_by_repo("facebook/react")), columns=["repo", "label", "title", "body"]).groupby("label").count()

,repo,title,body
label,,,
bug,974,974,973
feature,277,277,277
question,325,325,325
